# TriaMesh Geodesics

In [1]:
from lapy import TriaMesh

Load a triangle mesh of a flat square (OFF file).

In [2]:
T = TriaMesh.read_off("../data/square-mesh.off")
type(T).__name__

'TriaMesh'

In [3]:
# import plotting functions
import plotly
# plotly.offline.init_notebook_mode(connected=True)
from lapy.plot import plot_tria_mesh
import plotly.io as pio
pio.renderers.default = "sphinx_gallery"

We now plot the triangle mesh with a function overlay of the triangle quality. Note, that this is a function attached to the triangles, not the vertices, so it is piecewise flat.

In [4]:
q = T.tria_qualities()
plot_tria_mesh(T,plot_edges=True, tfunc=q)

## Laplace

Next, we check a few properties of eigenfunctions. For this we get the first few and also the stiffness matrix (A) and the lumped mass matrix (B, reduced to a diagonal), of which we can easily compute the inverse.

In [5]:
# compute first eigenfunction
from lapy import Solver
fem = Solver(T,lump=True)
eval, evec = fem.eigs()
vfunc = evec[:,1]

# also get A,B (lumped), and inverse of B (easy as it is diagonal due to lumping)
A, B = fem.stiffness, fem.mass
Bi = B.copy()
Bi.data **= -1

TriaMesh with regular Laplace-Beltrami
Solver: spsolve (LU decomposition) ...


The mass matrix B represents the inner product so that the integral of the product of two functions x and y over the whole mesh is x B y'. The lumped mass matrix that we use here is a simplified version, as all off-diagonal elements are added to the diagonal. The entries on the diagonal represent the vertex areas and their sum is the total area of the mesh. For our unit square it will be 1.

In [6]:
B.sum()

1.0

Let's see what happens when we apply the Laplace operator to an eigenfunction. Eigenfunctions are solutions to Delta f = - lambda f, so we should obtain a scaled version of f.

In [7]:
plot_tria_mesh(T,Bi*(A*vfunc),plot_edges=True)

This is the same as the corresponding eigenvalue times the eigenfunction.

In [8]:
plot_tria_mesh(T,eval[1]*vfunc,plot_edges=True)

Laplace is also defined as the -div(grad(f)). So first applying the gradient and then the divergence to an eigenfunction and then multiplying with inv(B) should yield the same result as above again. Note, that multiplying with inv(B) is necessary to get back from the integrated divergence to the original function.

In [9]:
from lapy.diffgeo import compute_gradient
from lapy.diffgeo import compute_divergence
grad = compute_gradient(T,vfunc)
divx = -compute_divergence(T,grad)
plot_tria_mesh(T,Bi*divx,plot_edges=True)

## Geodesics

Now we will replicate the idea of geodesics in heat, where first a heat diffusion is solved and massaged in the right way to yield an approximation of geodesics on the mesh. This also works on curved meshes, but for simplicity we keep using the square here. So let's start with computing the heat diffusion from the boundary (with default time factor m=1).

In [10]:
from lapy import heat
bvert = T.boundary_loops()
u = heat.diffusion(T,bvert,m=1)

TriaMesh with regular Laplace-Beltrami
Matrix Format now:  csc
Solver: spsolve (LU decomposition) ...


We show some of the level sets. Note, that they are not evenly spaced and get steeper closer to the boundary.

In [11]:
plot_tria_mesh(T,u,plot_edges=True,plot_levels=True)

Next step is to compute the gradient (vector field) of the heat diffusion function and normalize all vectors to unit length.

In [12]:
import numpy as np

# compute gradient of heat diffusion
tfunc = compute_gradient(T,u)

# normalize gradient
X = -tfunc / np.sqrt((tfunc**2).sum(1))[:,np.newaxis]
X = np.nan_to_num(X)

Then we get the integrated divergence of the normalized gradients.

In [13]:
divx = compute_divergence(T,X)

Finally, to obtain the distance function, we need to solve a Poisson equation. The solution can be shifted arbitrary, so we need to subtract the minimum, which should be along the boundary of the mesh.

In [14]:
# compute distance
from scipy.sparse.linalg import splu
useCholmod = True
try:
    from sksparse.cholmod import cholesky
except ImportError:
    useCholmod = False

fem = Solver(T,lump=True)
A, B = fem.stiffness, fem.mass

H=-A
b0=divx
        
# solve H x = b0
# we don't need the B matrix here, as divx is the integrated divergence 
print("Matrix Format now: "+H.getformat())
if useCholmod:
    print("Solver: cholesky decomp - performance optimal ...")
    chol = cholesky(H)
    x = chol(b0)
else:
    print("Solver: spsolve (LU decomp) - performance not optimal ...")
    lu = splu(H)
    x = lu.solve(b0)

# remove shift
x = x-min(x)

TriaMesh with regular Laplace-Beltrami
Matrix Format now: csc
Solver: spsolve (LU decomp) - performance not optimal ...


In short, the idea is to first get a function (heat diffusion) that flows from a point or from a set of points (boundary) through the mesh, then normalize its gradient, compute the divergence and finally step backward through the Laplace operator to find a function that has this normalized gradient. If we look at it, we actually notice that level sets are equally spaced now.

In [15]:
plot_tria_mesh(T,x,plot_edges=True,plot_levels=True)

Nice, but only an approximation.

In [16]:
# max distance (smoothed)
(max(x), np.sqrt(2)/2)

(0.60497856, 0.7071067811865476)

Instead of computing the solution ourselves, we could simply employ the standard Poisson solver on inv(B) times divx.

In [17]:
vf = fem.poisson(-Bi*divx)
vf = vf - min(vf)
plot_tria_mesh(T,vf,plot_edges=True,plot_levels=True)

Matrix Format now: csc
Solver: spsolve (LU decomposition) ...


This should give us the same result as what we had earlier.

In [18]:
max(abs(vf-x))

184.0

Or we can just call compute_geodesic_f which does all the work for us.

In [19]:
from lapy.diffgeo import compute_geodesic_f
gf = compute_geodesic_f(T,u)
plot_tria_mesh(T,gf,plot_edges=True,plot_levels=True)

TriaMesh with regular Laplace-Beltrami
Matrix Format now: csc
Solver: spsolve (LU decomposition) ...


And verify it is still the same result.

In [20]:
# again should be the same
max(abs(gf-x))

0.0

Similar to normalizing the gradients of a function, we can see if we can rotate it.

In [21]:
# testing if we can rotate the function
from lapy.diffgeo import compute_rotated_f
gf = compute_rotated_f(T,vf)
plot_tria_mesh(T,gf,plot_edges=True,plot_levels=True)

TriaMesh with regular Laplace-Beltrami
Matrix Format now: csc
Solver: spsolve (LU decomposition) ...


## Mean Curvature Mapping

To demonstrate the mean curvature mapping to a sphere, we need to have a closed mesh. It should not have too many flat regions (so not the cube) as there is no cuvature.

In [22]:
from lapy.diffgeo import tria_mean_curvature_flow
from lapy.plot import plot_tria_mesh
# load your mesh here and uncomment. The mesh should have not too many flat regions (not a cube)
#T = TriaIO.import_off("../data/???")
#T2 = tria_mean_curvature_flow(T)
#plot_tria_mesh(T2,plot_edges=True,plot_levels=True)